# Hibernate Example

We'll start by getting h2 database and Hibernate maven dependencies.

In [6]:
//DEPS com.h2database:h2:2.3.232
//DEPS org.hibernate.orm:hibernate-core:7.0.2.Final

JBangInfo{dependencies=2,resolvedDependencies=17,newResolvedDependencies=0}

Now lets define a `Book` entity.

In [7]:
import jakarta.persistence.*;

@Entity(name = "Book")
@Table(name = "books")
class Book {
    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    public Long id;

    public String title;
    public String author;

    @Override
    public String toString() {
        return "Book{" +
                "id=" + id +
                ", title='" + title + '\'' +
                ", author='" + author + '\'' +
                '}';
    }
}

Now we'll setup a Hibernate Configuration that has configuration for h2 database and include the `Book` entity.

In [8]:
import org.hibernate.jpa.HibernatePersistenceConfiguration;

var config = new HibernatePersistenceConfiguration("Jakarta Data Example")
.managedClasses(Book.class)
.property("hibernate.connection.url", "jdbc:h2:mem:test")
.property("hibernate.connection.username", "sa")
.property("hibernate.connection.password", "");


Finally we can build a Hibernate SessionFactory and run operations; in this example we use a `StatlessSession` as the simplest way of mapping data to from a database.

In [9]:

try (var sessionFactory = config.createEntityManagerFactory()) {
    var schemaManager = sessionFactory.getSchemaManager();
    schemaManager.drop(true);
    schemaManager.create(true);

    sessionFactory.inStatelessSession(session -> {
        var book = new Book();
        book.title = "The Hobbit";
        book.author = "J.R.R. Tolkien";
        IO.println(book);
        session.insert(book);

        IO.println("Book added " + book);

        session.createQuery("from java.lang.Object").list();
    });

    sessionFactory.inSession(session -> {
        session.createQuery("from Book").stream().forEach(IO::println);
    });


}

Book{id=null, title='The Hobbit', author='J.R.R. Tolkien'}
Book added Book{id=1, title='The Hobbit', author='J.R.R. Tolkien'}
Book{id=1, title='The Hobbit', author='J.R.R. Tolkien'}
